# Dependencies

In [1]:
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline
import threading


# Chatbot Configuration

In [2]:
# Configuration of embeddings and LLM models
embedding_model = "thenlper/gte-large"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Load vectorstore
vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embeddings)

# LLM Configuration
llm_model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)
hf_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=1024)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Conversational memory configuration
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# LangChain pipeline configuration
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory
)


Device set to use cpu


# Flask Configuration

In [3]:
# Flask server configuration
app = Flask(__name__)
CORS(app)

# Route for interaction with the chatbot
@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "No query provided"}), 400

    try:
        response = qa_chain({"question": query})
        return jsonify({"response": response["answer"]})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Route to display the frontend
@app.route("/")
def index():
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Chatbot</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background: #f0f0f5;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
            }
            .chat-container {
                width: 600px;
                background: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            }
            .messages {
                height: 300px;
                overflow-y: auto;
                border: 1px solid #ddd;
                border-radius: 8px;
                margin-bottom: 20px;
                padding: 10px;
                background: #f9f9f9;
            }
            .messages div {
                margin: 10px 0;
                padding: 8px;
                border-radius: 5px;
            }
            .user {
                background: #cce5ff;
                align-self: flex-end;
            }
            .bot {
                background: #d4edda;
                align-self: flex-start;
            }
            input[type="text"] {
                width: calc(100% - 80px);
                padding: 10px;
                border: 1px solid #ddd;
                border-radius: 5px;
            }
            button {
                padding: 10px 20px;
                border: none;
                border-radius: 5px;
                background: #007bff;
                color: white;
                cursor: pointer;
            }
            button:hover {
                background: #0056b3;
            }
        </style>
    </head>
    <body>
        <div class="chat-container">
            <div class="messages" id="messages"></div>
            <input type="text" id="userInput" placeholder="Type 'exit' or 'bye' to leave...">
            <button onclick="sendMessage()">Send</button>
        </div>

        <script>
            const messagesDiv = document.getElementById("messages");
            
            function initializeChat() {
                appendMessage("Hello! How can I assist you today?", "bot");
            }

            function appendMessage(message, sender) {
                const messageDiv = document.createElement("div");
                messageDiv.textContent = message;
                messageDiv.className = sender;
                messagesDiv.appendChild(messageDiv);
                messagesDiv.scrollTop = messagesDiv.scrollHeight;
            }

            async function sendMessage() {
                const userInput = document.getElementById("userInput").value;
                if (!userInput) return;

                appendMessage(userInput, "user");
                document.getElementById("userInput").value = "";

                if (userInput.toLowerCase() === "exit" || userInput.toLowerCase() === "bye") {
                appendMessage("Thank you for chatting with me. Goodbye!", "bot");
                return;
                }

                try {
                    const response = await fetch("http://127.0.0.1:5000/chat", {
                        method: "POST",
                        headers: { "Content-Type": "application/json" },
                        body: JSON.stringify({ query: userInput })
                    });
                    const data = await response.json();
                    if (data.response) {
                        appendMessage(data.response, "bot");
                    } else {
                        appendMessage("Error: " + (data.error || "Unknown error"), "bot");
                    }
                } catch (error) {
                    appendMessage("Error: Unable to connect to the server.", "bot");
                }
            }

            // Inicializar o chat com uma mensagem
            initializeChat();
        </script>
    </body>
    </html>
    """
    return render_template_string(html)

# Server Initialization

In [4]:
# Start the Flask server in a separate thread
def start_flask():
    app.run(host="127.0.0.1", port=5000, debug=False)

import threading
thread = threading.Thread(target=start_flask)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
